<a href="https://colab.research.google.com/github/Niyayeshnazerian/ML-Project/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio opencv-python scikit-learn numpy pandas


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

# دیتاست نمونه
data = {
    "age":[10,12,14,16,18,11,15,17],
    "height":[140,150,160,170,175,145,165,172],
    "arm":[120,125,130,135,140,122,132,138],
    "agility":[15,13,12,11,10,14,12,11],
    "reaction":[0.5,0.4,0.3,0.2,0.15,0.45,0.3,0.2],
    "gender":[0,0,1,1,1,0,1,1],  # 0: زن، 1: مرد
    "label":[0,0,1,1,1,0,1,1]   # 0: استعداد کم، 1: استعداد مناسب
}

df = pd.DataFrame(data)
X = df[["age","height","arm","agility","reaction","gender"]]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# ذخیره مدل
joblib.dump(model, "tennis_text_model.pkl")


['tennis_text_model.pkl']

In [3]:
import gradio as gr
import cv2
import numpy as np
import joblib

text_model = joblib.load("tennis_text_model.pkl")

def analyze_tennis(age, height, arm, agility, reaction, gender, video):
    # بخش متنی
    gender_num = 1 if gender=="مرد" else 0
    text_pred = text_model.predict([[age,height,arm,agility,reaction,gender_num]])[0]

    # بخش ویدیویی (Motion Detection ساده)
    cap = cv2.VideoCapture(video)
    prev = None
    motion_scores = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7,7), 0)
        if prev is not None:
            diff = cv2.absdiff(prev, gray)
            motion_scores.append(np.mean(diff))
        prev = gray
    cap.release()
    motion_score = np.mean(motion_scores) if motion_scores else 0
    video_pred = 1 if motion_score>5 else 0  # Threshold ساده برای امتیاز

    # ترکیب دو بخش
    final_score = text_pred + video_pred
    if final_score >= 2:
        return f"✅ استعداد مناسب برای تنیس (امتیاز: {final_score})"
    else:
        return f"⚠️ نیاز به تمرین بیشتر (امتیاز: {final_score})"


In [5]:
app = gr.Interface(
    fn=analyze_tennis,
    inputs=[
        gr.Slider(8,25,step=1,label="سن"),
        gr.Number(label="قد (cm)"),
        gr.Number(label="طول دو دست (cm)"),
        gr.Slider(10,25,label="زمان تست چابکی (ثانیه)"),
        gr.Slider(0.1,0.6,label="زمان عکس‌العمل (ثانیه)"),
        gr.Radio(["زن","مرد"],label="جنسیت"),
        gr.Video(label="ویدیوی فورهند")
    ],
    outputs="text",
    title="🎾 اپلیکیشن استعدادیابی تنیس "
)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://074d0185ee7f0c8521.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
